# Reconstructing 3D wireframes from Hesse normal form data

## Import Json file and load relevant libraries.

In [ ]:
#import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

rel_path = 'BoxData_All_SDs.json'

data_file = open(rel_path)

data = json.load(data_file)

## Normalization

First, normalize data and choose needed columns. Then, replace empty Values with None and convert column names. Last, values should all be numeric. 

In [ ]:
#Normalize data
df = json_normalize(data)

#Choose columns
cols = [col for col in df.columns if 'Fitted' in col]
df2 = df[cols]

#Replace empty with None
df3 = df2.replace(['empty'],[None])

#Convert column names (too long
df3.columns = df3.columns.str.replace('Fitted Planes.Plane','')

#Convert from string to numeric values
df4 = df3.convert_objects(convert_numeric=True)

In [ ]:
df.head(3)

In [ ]:
df4.head(3)

In [ ]:
len(data)
for x in range(len(data)):
    print('{0}:  {1}'.format(x,data[x]['ObjID']))

In [ ]:
data[18]

## Construct dictornary keys for every Hesse normal form 

Data is distance (1), normal vector (3), orientation (1), thus every five elements belong to one normal form. The key name is choosen from the dataframe, by splitting the string and taking only the first element.

In [ ]:
def varDictI(k):
    varDict = {}
    #len(df3.loc[0])
    for i in [0,5,10,15,20,25,30]:
        varDict['{0}{1}'.format(df4.loc[k].keys()[i].split('.')[0],0)] = [df4.loc[k][i],df4.loc[k][i+1],df4.loc[k][i+2],df4.loc[k][i+3],df4.loc[k][i+4]]
    return varDict

Create two lists. One, which contains all indexes where one or more entry of the dict is None and another where all entries are numbers.  

In [ ]:
#List with None values
iList = []
for i in range(len(df4)):
    if np.isnan([item for sublist in varDictI(i).values() for item in sublist]).any():
        iList.append(i)

# List withour None values    
iposList = []
for i in range(len(df4)):
    if not np.isnan([item for sublist in varDictI(i).values() for item in sublist]).any():
        iposList.append(i)

## Calculating corner points and plotting

The points of one corner can be calculated by the three intersecting planes, e.g. top, back, and east side. 
The Hesse normal forms of these sides constitute a system of equations for coordinates x,y,z which should have a unique solution. 
This system can be solved by the numpy package
```python
    a = np.array([[a0,a1],[a2,a3]])
    b = np.array([b1,b2])
    x = np.linalg.solve(a,b)
```  
where a is the 3x3 matrix of normal vectors, and b is the vector of distances. 

The calculated points are plotted with matplotlib as a scatter plot. Connecting lines are plotted by calculating the coordinates x,y,z of the lines connecting two corners. 

To plot, the function idealSundial takes the index as input. If the chosen index is not in the positive list, a Value Error is raised. 

*Under development* Calculation of intersection points of three planes give wrong results compared with json data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

x=18
dataframe=df4
check = {}
checkList = []
for key in data[x]['Intersection Points'].keys():
    subList = []
    for key2 in data[x]['Intersection Points'][key].keys():
        subList.append(data[x]['Intersection Points'][key][key2])
    checkList.append(subList)
    #print(checkList)
    check['{0}'.format(key[23:])] = subList
    
#def idealSundial(dataframe, x=iposList[0]):
if x in iList:
    raise ValueError('Coordinate list for i = %s contains nan Values.' % x)

varDict = {}
#len(df3.loc[0])
for i in [0,5,10,15,20,25,30]:
    varDict['{0}{1}'.format(dataframe.loc[x].keys()[i].split('.')[0],0)] = [dataframe.loc[x][i],dataframe.loc[x][i+1],dataframe.loc[x][i+2],dataframe.loc[x][i+3],dataframe.loc[x][i+4]]

nBa = varDictI(x)['Back0']
nBo = varDictI(x)['Bottom0']
nTo = varDictI(x)['Top0']
nE  = varDictI(x)['East0']
nW  = varDictI(x)['West0']
nFL = varDictI(x)['FrontLP0']
nFU = varDictI(x)['FrontUP0']

# Solve system of equations for every corner point
XList = []
for x in [[nFL,nBo,nW],[nFU,nTo,nW],[nFU,nTo,nE],[nFL,nBo,nE],
          [nBa,nBo,nW],[nBa,nTo,nW],[nBa,nTo,nE],[nBa,nBo,nE],[nFU,nFL,nW],[nFU,nFL,nE]]:
    a = np.array([[x[0][1],x[0][2],x[0][3]],
                 [x[1][1],x[1][2],x[1][3]],
                 [x[2][1],x[2][2],x[2][3]]])
    b = np.array([x[0][0]*x[0][4],x[1][0]*x[1][4],x[2][0]*x[2][4]])
    x = np.linalg.solve(a,b)
    XList.append(x)


# Create the figure
#fig2 = plt.figure()
fig2 = plt.figure(figsize=plt.figaspect(0.5))
fig2.suptitle('A tale of 2 subplots')

# Add an axes
ax2 = fig2.add_subplot(1,2,1,projection='3d')

# Add points to plot
for x in XList:
    ax2.scatter(x[0],x[1],x[2])

# Add connecting lines
#for a, b in [[XList[0],XList[3]],[XList[0],XList[4]],[XList[0],XList[8]],
#             [XList[1],XList[2]],[XList[1],XList[5]],[XList[1],XList[8]],
#             [XList[2],XList[6]],[XList[2],XList[9]],
#             [XList[3],XList[9]],[XList[3],XList[7]],
#             [XList[4],XList[5]],[XList[4],XList[7]],
#             [XList[5],XList[6]],
#            [XList[6],XList[7]],[XList[8],XList[9]]]:
#    x = np.linspace(a[0], b[0], 100)
#    y = np.linspace(a[1], b[1], 100)
#    z = np.linspace(a[2], b[2], 100)
#    ax2.plot(x, y, z)

ax3 = fig2.add_subplot(1,2,2,projection='3d')

arr = []
for key in check.keys():
    ls = [float(x) for x in check[key]]
    arr.append(ls)

for y in arr:
    ax3.scatter(y[0],y[1],y[2],color='red')

#limit plotting range
#ax2.set_xlim3d(-200, 200)
#ax2.set_ylim3d(-200, 200)
#ax2.set_zlim3d(-200, 200)
ax2.set_xlabel('X axis')
ax2.set_ylabel('Y axis')
ax2.set_zlabel('Z axis')
#return plt.show()


In [ ]:
# Solve system of equations for every corner point
XList = []
for x in [[nW,nBo,nFL]]:
    print(x[0])
    a = np.array([[x[0][1],x[0][2],x[0][3]],
                 [x[1][1],x[1][2],x[1][3]],
                 [x[2][1],x[2][2],x[2][3]]])
    b = np.array([x[0][0]*x[0][4],x[1][0]*x[1][4],x[2][0]*x[2][4]])
    x = np.linalg.solve(a,b)
x

In [ ]:
idealSundial(df4,45)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

# Raises index error...
#interact(idealSundial,x=iposList,dataframe=fixed(df4))

In [ ]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.gca(projection='3d')

for pl in [nE,nW,nBo,nTo,nBa,nFL,nFU]:
    x,y,z = (0,0,0)
    u,v,w = (pl[0]*pl[1]*pl[4],pl[0]*pl[2]*pl[4],pl[0]*pl[3]*pl[4])
    ax.quiver(x,y,z,u,v,w,pivot='tail')

ax.set_xlim3d(-2, 2)
ax.set_ylim3d(-2, 2)
ax.set_zlim3d(-2, 2)
plt.show()

## Alternative approach 

Alternatively, the package sympy allows to calculate intersections of planes, defined by Hesse normal forms. These intersections are lines in 3D, which again have intersections with other lines, giving rise to the 3D coordinates of the corners. 

However, this approach seems not robust and there should be an error in the definition of the planes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from sympy import Point, Point3D, Line, Line3D, Plane


%matplotlib notebook

X = np.linspace(-200, 200, 100)            
Y = np.linspace(-200, 200, 100)
xx, yy = np.meshgrid(X,Y)

#Define sets of Hesse normal form data for each site. 
i=0

if i in iList:
    raise ValueError('Coordinate list for i = %s contains nan Values.' % i)

nBa = varDictI(i)['Back0']
nBo = varDictI(i)['Bottom0']
nTo = varDictI(i)['Top0']
nE  = varDictI(i)['East0']
nW  = varDictI(i)['West0']
nFL = varDictI(i)['FrontLP0']
nFU = varDictI(i)['FrontUP0']

    
# Find all z coordinates, that fullfill Hesse normal form
zBa =  (-nBa[1]  * xx - nBa[2]  * yy - nBa[0]/100) * 1. / nBa[3]
zBo =  (-nBo[1]  * xx - nBo[2]  * yy - nBo[0]/100) * 1. / nBo[3]
zTo  = (-nTo[1]  * xx - nTo[2]  * yy - nTo[0]/100) * 1. / nTo[3]
zE  =  (- nE[1]  * xx -  nE[2]  * yy -  nE[0]/100) * 1. / nE[3]
zW  =  (- nW[1]  * xx -  nW[2]  * yy -  nW[0]/100) * 1. / nW[3]

#Test norm?! 

#Define planes in sympy using given Hesse normal form
ba = Plane(Point3D(nBa[0], 0, 0), normal_vector=(nBa[1], nBa[2], nBa[3]))
to = Plane(Point3D(nTo[0], 0, 0), normal_vector=(nTo[1], nTo[2], nTo[3]))
bo = Plane(Point3D(nBo[0], 0, 0), normal_vector=(nBo[1], nBo[2], nBo[3]))
we = Plane(Point3D(nW[0], 0, 0), normal_vector=(nW[1], nW[2], nW[3]))
ea = Plane(Point3D(nE[0], 0, 0), normal_vector=(nE[1], nE[2], nE[3]))
flp = Plane(Point3D(nFL[0],0, 0), normal_vector=(nFL[1], nFL[2], nFL[3]))
fup = Plane(Point3D(nFU[0],0, 0), normal_vector=(nFU[1], nFU[2], nFU[3]))

#Calculate intersections of three planes, gives points of corners in 3D
x1 = flp.intersection(bo)[0].intersection(we)[0]
x2 = fup.intersection(to)[0].intersection(we)[0]
x3 = fup.intersection(to)[0].intersection(ea)[0]
x4 = flp.intersection(bo)[0].intersection(ea)[0]
x5 = ba.intersection(to)[0].intersection(we)[0]
x6 = ba.intersection(to)[0].intersection(ea)[0]
x7 = ba.intersection(bo)[0].intersection(ea)[0]
x8 = ba.intersection(bo)[0].intersection(we)[0]
x9 = flp.intersection(fup)[0].intersection(we)[0]
x10 = flp.intersection(fup)[0].intersection(ea)[0]

# Create the figure
fig = plt.figure()

# Add an axes
ax = fig.add_subplot(111,projection='3d')

# plot the surface
#ax.plot_surface(xx, yy, zBa,alpha=0.2)
#ax.plot_surface(xx, yy, zBo,alpha=0.2)
#ax.plot_surface(xx, yy, zTo,alpha=0.2)
#ax.plot_surface(xx, yy, zE,cmap=cm.BuPu,alpha=0.2)
#ax.plot_surface(xx, yy, zW,cmap=cm.BuPu,alpha=0.2)

#Points are given as tuples of sympy objects, thus .args[i]
ax.scatter(x1.args[0], x1.args[1], x1.args[2],  color='red')
ax.scatter(x2.args[0], x2.args[1], x2.args[2],  color='red')
ax.scatter(x3.args[0], x3.args[1], x3.args[2],  color='red')
ax.scatter(x4.args[0], x4.args[1], x4.args[2],  color='red')
ax.scatter(x5.args[0], x5.args[1], x5.args[2],  color='red')
ax.scatter(x6.args[0], x6.args[1], x6.args[2],  color='red')
ax.scatter(x7.args[0], x7.args[1], x7.args[2],  color='red')
ax.scatter(x8.args[0], x8.args[1], x8.args[2],  color='red')
ax.scatter(x9.args[0], x9.args[1], x9.args[2],  color='red')
ax.scatter(x10.args[0], x10.args[1], x10.args[2],  color='red')


#Linspace not working with sympy objects, convert to float
x = np.linspace(float(x1.args[0]), float(x4.args[0]), 10)
y = np.linspace(float(x1.args[1]), float(x4.args[1]), 10)
z = np.linspace(float(x1.args[2]), float(x4.args[2]), 10)
ax.plot(x, y, z)
    
ax.set_xlim3d(90, 130)
ax.set_ylim3d(-5, 5)
ax.set_zlim3d(-5, 5)

plt.show()